<a href="https://colab.research.google.com/github/DataSciBurgoon/2024_ASCCT_CE/blob/main/ASCCT_SimIntro_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Install packages, as needed.
# # Only need to do this once in your Google Colab session.
# install.packages("lattice")
# install.packages("mvtnorm")

In [ ]:
# Load packages once installed
library(lattice)
library(mvtnorm)

# 1. Introduction

One of the advantages of having computing power and statistical softwares is that we can run simulation experiments to evaluate performance of our models, the robustness and limitations of different procedures, or simply to further understand some concepts.

Here, we will consider a few simple examples and will set up simulations experiments to illustrate and explore some relevant concepts.

First, we need learn an important tool that is very useful when conducting simulation exercises so we can replicate and share our results.

Say that we want to generate 5 observations from a Normal distribution

In [ ]:
# This function is generating 5 random observations from a Normal distribution
# with mean 0 and standard deviation 1
rnorm(n = 5, mean = 0, sd = 1)

# Here we are generating 5 new random observations from a Normal distribution
# with mean 0 and standard deviation 1
rnorm(5, 0, 1)

Note that when we run the code for the second time, we get a different set of 5 observations from a Normal distribution.

This is usually ok, because we want to generate 5 random observations. However, if you would like to share your code or reproduce your own results, you won't be able to do so because every time we run that line we will get 5 (probably) different random observations

In [ ]:
rnorm(5, 0, 1)

There is a way to "fix" this problem, using the `set.seed(.)` function in R

In [ ]:
# In the function we are using "512" as the seed, but we can pick any other
# integer
set.seed(512)
rnorm(5, 0, 1)

set.seed(512)
rnorm(5, 0, 1)

We note now that we have generated two identical sets of 5 observations from a normal distribution.

If we run the `rnorm(.)` function again, we will get a different set. But if we set the seed 512 again, we will be able to reproduce the results above

In [ ]:
# Generate 5 new random observations
rnorm(5,0,1)

# Now we get the same numbers we got before
set.seed(512)
rnorm(5, 0, 1)

# 2. Sampling distributions and transformations

Let's start by generating observations from two populations $X$ and $Y$ and plot a histogram of the observations.

In [ ]:
# Set a seed so everyone's "random" results will be the same.
set.seed(159823)

# Set the sizes of our two samples
n1 <- 1000
n2 <- 1000

# Generate samples from our two populations
X <- 100 * rgamma(n1, shape = 0.58, scale= 2 / 0.58)
Y <- 100 * exp(rnorm(n2, log(2) - 1, sqrt(2)))

# Plot the distributions of our samples
par(mfrow=c(1,2))
hist(X, breaks=seq(0, 6200, 100), main="Histogram for X", xlab="x-values")
hist(Y, breaks=seq(0, 6200, 100), main="Histogram for Y", xlab="y-values")
par(mfrow=c(1,1))

Suppose that we are interested in performing a test to compare the means of these two populations. Can we just perform a t-test? Or should we consider a transformation?

In [ ]:
# t-test results on our original scale
t.test(X, Y)

# t-test after log transformation
t.test(log(X), log(Y))

Even though the original populations are skewed, both tests are valid, but different. In the first test the null hypothesis is
$$H_0: \mu_x=\mu_y,$$
where $\mu_x,\mu_y$ are the population means of the populations for $X$ and $Y$ respectively.

In the second test the null hypothesis is
$$H_0: \mu_{log(x)}=\mu_{log(y)},$$
where $\mu_{log(x)},\mu_{log(y)}$ are the population means of the log transformed variables $X$ and $Y$ respectively.

# 3. Confidence intervals

Let's recall from our stats courses how to construct a confidence interval for the population mean. Say that we observe draw $n$ observations from a $N(\mu,1)$ distributed population, where $\mu$ is unknown. That is, we don't know what the average value for some variable of interest is with a population, but we'd like to use a sample to gather some information about what the population mean might be.

We can construct a $(1-\alpha)\times 100\%$ confidence interval for $\mu$, based on our observations using the formula
$$
\bar{x} \pm z_{\alpha/2} \times \frac{\sigma}{\sqrt{n}}
$$
Some people misinterpret a (say) 95% confidence interval as a 95% probability that the the true mean will land in the resulting interval. We will conduct a short simulation experiment to help us understand the concept of "confidence level".

First, we set up our simulation parameters

In [ ]:
# Population mean (Considered "unknown" even though we know it's equal to 1)
mu <- 1

# Population standard deviation (known)
sigma <- 1

# Sample size in one run of the experiment (i.e. number of observations we
# have)
n.sample <- 10

# Number of times we will repeat the experiment
N.rep <- 1000

# alpha for the desired (1-alpha)x100% Confidence level
alpha <- 0.05

In this set up `n.sample` is the number of observations we will draw from the $N(\mu,1)$ distribution in one repetition of the experiment (this would correspond to the sample size we would have in a real experiment).

On the other hand, `N.rep` corresponds to the number of times we want to repeat the entire experiment. That is, if we went to the lab over and over again, and each time we obtain 10 observations from our experiment (this is what we can't do in a real situation, but can simulate using a computer).

Note that in a "real" experiment the true value of $\mu$ would be unknown. But here we need to assign a value so we can run the simulations. Still, we will perform all the calculations pretending we don't know what the value of $\mu$ is to check if we can get the expected results.

Now, because the observations are drawn from a Normal distribution, we know that the sampling distribution of the sample mean is also Normal. More specifically, we know that

$$
\bar{X}\sim N\big(\mu,\sigma^2/n\big)
$$

So we can generate the `N.rep` repetitions of the experiment using

In [ ]:
# Sample means generated from sampling distribution
x.bar <- rnorm(N.rep, mean = mu, sd = sigma / sqrt(n.sample))
# Read the first few observations
head(x.bar)

Now we can construct the confidence intervals for each one of the replications of the experiment.

In [ ]:
# z-value for confidence intervals
z <- qnorm(alpha / 2, mean = 0, sd = 1, lower.tail=FALSE)

# Confidence interval lower bound
low.bound <- (x.bar - z * sigma / sqrt(n.sample))

# Confidence interval upper bound
up.bound <- (x.bar + z * sigma / sqrt(n.sample))

# Data frame to see the intervals in column-form
CI <- data.frame(low.bound,up.bound)

# Read the first few observations
head(CI)

Finally, we need to check how many of these intervals contain the true value of $\mu$. Since we know that $\mu=1$ (At least for our simulation scenario) we should obtain approximately $(1-\alpha)\times 100\%$ of the intervals we constructed containing the true value of $\mu$.

In [ ]:
# Check if resulting interval contains the true parameter
contains <- (mu >= low.bound & mu <= up.bound)

# Observed proportion of intervals that cover the true value of the parameter
sum(contains) / N.rep

# 4. Uncertainty in regression models

Consider now a similar experiment in the context of linear regression. Fist we will generate observations from a true model and we will fit the linear regression line to the generated data.

In [ ]:
# This is the explanatory variable, created using n = 12 random values # uniformly distributed between min = 15 and max = 25.
X1 <- runif(n = 12, min = 15, max = 25)

# This is the response variable following the true model with intercept 5 and slope 0.5.
# The term "rnorm(length(X1), mean = 0, sd = 2)" is generating random errors
Y <- 5 + 0.5 * X1 + rnorm(length(X1), mean = 0, sd = 2)

# Here we are fitting the model to the generated data
fit <- lm(Y ~ X1)

# Are the estimated coefficients any close to the true values?
summary(fit)

We can plot the observations, fitted line and true line, so we can visualize the situation

In [ ]:
plot(x = X1, y = Y,
     type="p", pch=20, col="blue",
     xlim=c(0, 25), ylim=c(0, 25),
     xlab = "X",ylab = "Y", main="")
abline(fit, col = "blue", lwd = 1.5)
abline(a = 5, b = 0.5, col = "red", lwd = 1.5)

Now, we can construct the confidence intervals for the mean response

In [ ]:
# We will construct intervals at each one of these points
# so we can plot them later as a "confidence band" surrounding the fitted line
newx1 <- seq(0, 25, by=0.05)
conf_interval <- predict(fit,
                         newdata=data.frame(X1 = newx1),
                         interval="confidence",
                         level = 0.95)

Let's add the confidence intervals to the plot

In [ ]:
plot(x = X1, y = Y,
     type="p", pch=20, col="blue",
     xlim=c(0, 25), ylim=c(0, 25),
     xlab = "X",ylab = "Y", main="")
abline(fit, col = "blue", lwd = 1.5)
abline(a = 5, b = 0.5, col = "red", lwd = 1.5)

# This will plot the lower band
lines(newx1, conf_interval[,2], col="blue", lty=2)

# This will plot the upper band
lines(newx1, conf_interval[,3], col="blue", lty=2)

Now we can obtain the prediction intervals for a "future" response

In [ ]:
pred_interval <- predict(fit,
                         newdata=data.frame(X1=newx1),
                         interval="prediction",
                         level = 0.95)

and add the prediction bands to the plot

In [ ]:
plot(x = X1, y = Y,
     type="p", pch=20, col="blue",
     xlim=c(0, 25), ylim=c(0, 25),
     xlab = "X",ylab = "Y", main="")
abline(fit, col = "blue", lwd = 1.5)
abline(a = 5, b = 0.5, col = "red", lwd = 1.5)

# Add the confidence interval
lines(newx1, conf_interval[,2], col="blue", lty=2)
lines(newx1, conf_interval[,3], col="blue", lty=2)

# Add the prediction intervals
lines(newx1, pred_interval[,2], col="darkgreen", lty=2)
lines(newx1, pred_interval[,3], col="darkgreen", lty=2)

We observe that the estimated value of the slope coefficient does not match with the true value of the slope (which is expected), but perhaps a confidence interval for $\beta_1$ will capture the true value of the coefficient.

We can run simulations to check the performance of such intervals and explore the connections with other concepts, such as P-values. First, we set up the simulation framework

In [ ]:
# Set up the simulation parameters

# Number of repeated "experiments"
M <- 1000

# Each experiment will be of sample size ...
n.sample <- 12

# Additional confidence interval parameters
sigma <-  2
alpha <- 0.05

# Some empty vectors to "store" the results of our M experiments.
pvalue <- vector("numeric", M)
beta1 <- vector ("numeric",M)
coverage <- vector("logical",M)

Once we set up the simulation parameters, we can generate observations, construct the intervals and look at P-values

In [ ]:
# Using a for loop to allow us to repeat our fairly complex experiments
for(i in 1:M) {

  # Generate our explanatory and response values
  X <- runif(n.sample,min = 15, max = 25)
  Y <- 5 + 0.5 * X + rnorm(length(X), mean = 0, sd = sigma)

  # Generate our "critical value" for the confidence intervals.
  t <- qt(alpha / 2, n.sample - 1, lower.tail = FALSE)

  # Fit the regression model
  model <- lm(Y ~ X)

  # Extract estimated coefficients, store them in our storage vector
  beta1[i] <- summary(model)$coeff[2,1]

  # Extract corresponding p-value, store it in our storage vector
  pvalue[i] <- summary(model)$coeff[2,4]

  # Check if reulting interval contains the true parameter
  coverage[i] <- (0.5 >= summary(model)$coeff[2,1] -
                    t*summary(model)$coeff[2,2] & 0.5 <=
                    summary(model)$coeff[2,1] +
                    t*summary(model)$coeff[2,2])
}

When the simulations are done we can use them evaluate the performance of the procedures using different metrics and visualization tools

In [ ]:
# Construct histogram
# What is the histogram of the estimated coefficients telling you?
hist(beta1, breaks =  50)

# Observed proportion of intervals that cover the true value of the parameter
# What do you think of the observed coverage of the intervals?
sum(coverage) / M

# Check if P-value is not "significant" (Observed proportion of rejections)
# What do you think of the proportion of times you fail to reject the
# hypothesis
# H_0 : beta_1 = 0?
fail.reject <- (pvalue > alpha)
sum(fail.reject) / M

# 5. Multicollinearity

When evaluating a model of the form
$$
\mu\{Y|\mathbf{X}\}=\beta_0+\beta_1X_1+\beta_2X_2+\ldots+\beta_pX_p
$$
it is not uncommon to look at the P-values of the individual coefficients to determine which variables are "important" by assessing significance.

However, the precision in estimating a particular regression coefficient may be affected when several explanatory variables are included in the model.

This loss of precision is expressed in usually large values for the standard errors of the estimated coefficients due to strong correlations among the explanatory variables in the model.

As a result, we might observe that some variables are not significant simply, because the the standard errors are too big.

## 5.1 Example

Let's run a short simulation example to understand multicollinearity:

- Construct a set of 3 explanatory variables $X_1, X_2, X_3$, so that $X_1, X_2$ are correlated, but $X_1, X_3$ are independent.
- Construct a response variable $Y$, so that $Y$ is associated with $X_1$, but not with $X_2, X_3$.

In [ ]:
# Construct our explanatory variables.
X1 <- runif(n = 50, min = 0, max = 20)
X2 <- 5 + 0.5 * X1 + rnorm(length(X1), mean = 0, sd = 0.025)
X3 <- runif(n = 50, min = 0, max = 20)

# Construct our response variable.
Y <- 10 + 15 * X1 + rnorm(length(X1), mean = 0, sd = 25)

Looking at the correlations among the predictors we obtain

In [ ]:
# Correlation between X1 and X2
cor(X1,X2)

# Correlation between X1 and X3
cor(X1,X3)

# Plot two plots, side by side
par(mfrow=c(1,2))

# High correlation variables scatterplot
plot(x = X1, y = X2,
     pch = 19, col = "blue",
     main= "High correlation",
     xlab="X2 vs X1")

# Low correlation variables scatterplot
plot(x = X1, y = X3,
     pch = 19, col = "blue",
     main = "Low correlation",
     xlab ="X3 vs X1")

# Reset so plots show up on their own
par(mfrow=c(1,1))

If we fit the model
$$\mu(Y|X_1,X_2)=\beta_0+\beta_1X_1+\beta_2X_2,$$
we should observe that $X_1$ is significant, but $X_2$ is not. However, because of the correlation between $X_1$ and $X_2$ we obtain

In [ ]:
# Fit the model with multicollinearity
mcol.yes <- lm(Y ~ X1 + X2)
summary(mcol.yes)

cloud(Y ~ X1 + X2,
      scales =  list(arrows = FALSE),
      pch=19,
      xlim = c(0, 20), ylim = c(0, 20), zlim = c(0, 350),
      main="Multicollinearity Yes")

On the other hand, when we fit the model $$\mu(Y|X_1,X_3)=\beta_0+\beta_1X_1+\beta_3X_3,$$
where there is no correlation bewteen $X_1,X_3$ we obtain


In [ ]:
# Fit the model without multicollinearity
mcol.no <- lm(Y~X1+X3)
summary(mcol.no)

cloud(Y ~ X1+ X3,
      scales = list(arrows = FALSE),
      pch=19,
      xlim = c(0, 20), ylim = c(0, 20), zlim = c(0, 350),
      main="Multicollinearity No")

Note that multicollinearity is affecting the standard errors for the estimated coefficients for $X_1$ in each model  

In [ ]:
# Look a standard errors in the model with multicollinearity
summary(mcol.yes)$coef

# Look a standard errors in the model without multicollinearity
summary(mcol.no)$coef

If the standard errors are large, then the t-values are small. And if the t-values are small, the P-values are large, suggesting the corresponding variables are not significant.

## 5.2 Why multicollinearity can be an issue?

- The least squares estimates of the regression coefficients will have big standard errors.
- The corresponding t-values will be small, and the P-values will be large (suggesting no significant effects even if there are).
- Because of the loss of precision in estimation adding/deleting a few
observations can have huge effects on the parameter estimates, making the interpretation difficult.

Note that all these are problematic when your model objective is to identify "relevant variables".

But, even under multicollinearity, your parameter estimates will still be unbiased and your model can offer a good fit.

We can use graphical tools to assess the correlation between variables, like a matrix of scatterplots.

In [ ]:
pairs(~ Y + X1 + X2 + X3, col = "blue")

# 6. Monte Carlo Simulations

Let's finish with some Monte Carlo simulations to estimate the prediction uncertainty in the models we used in the previous example.

Monte Carlo simulations typically consists of generating observations based on known distributions that we assume to be truth for our models.

In the previous sections we considered two different models of the form
$$
\mu\{Y|X_1, X_2=\beta_0+\beta_1X_1+\beta_2X_2\quad \text{(with multicol.)}
$$
and
$$
\mu\{Y|X_1, X_3=\beta_0+\beta_1X_1+\beta_3X_3\quad \text{without multicol.}
$$

And Suppose we want to estimate the uncertainty in the prediction of the response $Y$ when $X_1=X_2=X_3=10$.

In [ ]:
# Fix values of X1 - X3
X1_new = 10
X2_new = 10
X3_new = 10

## 5.1 Multicollinearity model (wrong approach)
Since the estimated coefficients have uncertainties (standard errors), a first approach could be to generate values for these coefficients using a known distribution and see how the error propagates throughout the model.

We get beta coefficients and standard errors

In [ ]:
# beta coefficients
B0 = mcol.yes$coefficients["(Intercept)"]
B1 = mcol.yes$coefficients["X1"]
B2 = mcol.yes$coefficients["X2"]

# beta standard errors
std.errors = sqrt(diag(vcov(mcol.yes)))

B0_se = std.errors["(Intercept)"]
B1_se = std.errors["X1"]
B2_se = std.errors["X2"]

Then simulate coefficients and response values "independently" according to a Normal distribution, based on the assumptions of the model

In [ ]:
# Perform N.sims simulations
N.sims = 1000

# And generate the coefficients according to our previous model's estiamtes.
B0_sims <- rnorm(N.sims, mean = B0, sd = B0_se)
B1_sims <- rnorm(N.sims, mean = B1, sd = B1_se)
B2_sims <- rnorm(N.sims, mean = B2, sd = B2_se)

# For each simulated value we produce a predicted value for the response using
# the model.
y_sims = B0_sims + X1_new * B1_sims + X2_new * B2_sims

Looking at the histogram of the simulated responses we obtain

In [ ]:
hist(y_sims, breaks = 50)

We can look at the 2.5th and 97.5th percentiles to obtain a prediction interval

In [ ]:
# Prediction interval lower bound
quantile(y_sims, p = c(0.025, 0.975))

predict(mcol.yes,
        newdata = list(X1 = X1_new, X2 = X2_new),
        interval = "confidence",
        level = 0.95)

predict(mcol.yes,
        newdata = list(X1 = X1_new, X2 = X2_new),
        interval = "prediction",
        level = 0.95)

We observe that our prediction was far from what we get using the "predict" function. Could this be due to multicollinearity?

## 5.2 No Multicollinearity model (wrong approach)
We repeat the same exercise , but now using the model without multicollinearity.

We get beta coefficients and standard errors

In [ ]:
# beta coefficients
B0 = mcol.no$coefficients["(Intercept)"]
B1 = mcol.no$coefficients["X1"]
B3 = mcol.no$coefficients["X3"]

# beta standard errors
std.errors = sqrt(diag(vcov(mcol.no)))

B0_se = std.errors["(Intercept)"]
B1_se = std.errors["X1"]
B3_se = std.errors["X3"]

Now we generate simulated values

In [ ]:
# Perform N.sims simulations
N.sims = 1000

# And generate the coefficients according to our previous model's estiamtes.
B0_sims <- rnorm(N.sims, mean = B0, sd = B0_se)
B1_sims <- rnorm(N.sims, mean = B1, sd = B1_se)
B3_sims <- rnorm(N.sims, mean = B3, sd = B3_se)

# For each simulated value we produce a predicted value for the response using
# the model.
y_sims = B0_sims + X1_new * B1_sims + X3_new * B3_sims

Look at histogram

In [ ]:
hist(y_sims, breaks = 50)

Still no good.

In [ ]:
# Prediction interval using simulated response values
quantile(y_sims, p = c(0.025, 0.975))

# Using predict
predict(mcol.no,
        newdata = list(X1 = X1_new, X3 = X3_new),
        interval = "prediction",
        level = 0.95)

Comparing with the values produced with the "predict" function, we are still way off. So the issue is NOT due to multicollinearity.

## The proper simulation approach

The problem is that in the previous approach we did not take into account the correlation structure between the estimated coefficients. To do so, we will need to use the package "MVNorm", which we loaded at the top of this notebook document.

Now, we repeat our previous exercise with the "No Multicollinearity" model first:

In [ ]:
N.sims = 1000
coef_samples = rmvnorm(N.sims,
                       mean = coef(mcol.no),
                       sigma = vcov(mcol.no))

y_without <- coef_samples %*% matrix(c(1, 10, 10), ncol = 1)
yp_without = y_without + rnorm(N.sims, sd = 25)

And we look at the interval based on the quantiles of the simulated values

In [ ]:
# CI
quantile(y_without, p = c(0.025, 0.975))
predict(mcol.no,
        newdata = list(X1 = 10, X3 = 10),
        interval = "confidence",
        level = 0.95)

# PI
quantile(yp_without, p = c(0.025, 0.975))
predict(mcol.no,
        newdata = list(X1 = 10, X3 = 10),
        interval = "prediction",
        level = 0.95)

Observe there is a difference now between the confidence intervals and prediction intervals we obtain with the prediction. Can you see where is that difference in the code?

Repeating the exercise for the model "With Multicollinearity", we obtain now

In [ ]:
coef_samples = rmvnorm(N.sims,
                       mean = coef(mcol.yes),
                       sigma = vcov(mcol.yes))

y_with = coef_samples %*% matrix(c(1, 10, 10), ncol = 1)
yp_with = y_with + rnorm(N.sims, sd = 25)

In [ ]:
# CI
quantile(y_with, p = c(0.025, 0.975))
predict(mcol.yes,
        newdata = list(X1 = 10, X2 = 10),
        interval = "confidence",
        level = 0.95)

# PI
quantile(yp_with, p = c(0.025, 0.975))
predict(mcol.yes,
        newdata = list(X1 = 10, X2 = 10),
        interval = "prediction",
        level = 0.95)

We observe that we can obtain good results, but we need to be very careful in the way we implement the Monte Carlo simulation.